# APIs para GPTs e outros LLMs no Python

## Por quê usar LLMs no Python:
1) Automatização
2) Controle
3) Escala 

## O que esse treinamento deve proporcionar
1) Configuração do ambiente: como criar e onde salvar a chave da API, como estabelecer limites de gastos
2) Primeiros passos: fazendo uma requisição básica
3) Trabalhando em escala: como utilizar batchs (síncrono e assíncrono)
4) Ganhando complexidade: use case Reclame Aqui 

Observações: utilizar YAML 
agents/
    - mapeador de tendências.yaml
    - comparador_entre_trend_e_macrotrend.yaml
    - ...

yaml:
    - modelo de API (ex: gpt4o)
    - temperatura
    - outros parâmetros
    - sys

## 1) Configurando o ambiente 

Application Programming Interface (APIs) são como pontes padronizadas para que dois sistemas conversem com segurança e rapidez. Na prática, serve para que possamos extrair funcionalidades de aplicações sem que seja necessário entendermos o que tem por trás da aplicação, apenas como nos conectar. APIs envolvem algumas regras, como autenticação, limites de pedidos e horário de funcionamneto.

No caso de LLMs, a API serve para utilizarmos modelos sem que precisemos de todo a aplicação web. No geral, o processo é parecido para quase todos os provedores (OpenAI, Google, Anthropic). Diferente da utilização tradicional em que se paga uma mensalidade, na utilização de APIs se paga por tokens utilizados (input/output). 

### Criando uma chave

O processo de criação é bem simples. Para a OpenAI, LLM que estaremos utilizando principalmente nesse treinamento, você pode criar sua própria chave aqui https://platform.openai.com/api-keys. Na maioria dos casos, você só consegue ler a chave completa quando cria, e após isso ela fica em grande parte oculta por segurança.

A chave terá um formato similar a esse: "sk-proj-XlTrIjLDJ...."

### Onde armazenar sua chave
É importante que sua chave fique armazenada em um local seguro. Para isso, normalmente utilizamos um arquivo ".env". Ele é onde você armazenam variáveis que dependem da máquina ou que não devem ser salvas em seu .git ou em locais públicos. O .env não é 100% seguro mas é bom para variáveis específicas da máquina ou usuário. 

No caso de utilizarmos .git em algum projeto, é recomendado deixar o .env dentro do .gitignore para evitarmos que essas variáveis sejam enviadas ao repositório.

A Mirow possui chaves para utilização geral.

## 2) Realizando requisições 
Vamos fazer uma requisição básica a OpenAI

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Carrega as variáveis do arquivo .env
load_dotenv()

# Inicializa o cliente
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Requisição
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",  # Instruções de alto nível para o modelo (tom, formato, regras)
            "content": "Você é um especialista em Python e análise de dados. Responda de forma clara e didática.",
        },
        {
            "role": "user",  # Pergunta ou comando do usuário
            "content": "Explique o que são APIs e por que são importantes para trabalhar com LLMs.",
        },
        {
            "role": "assistant",  # Resposta já geradas do assistente (modelo)
            "content": "APIs são interfaces que permitem comunicação entre sistemas diferentes de forma padronizada.",
        },
        {
            "role": "user",  # Nova pergunta ou comando do usuário
            "content": "Agora me dê um exemplo prático de uso.",
        },
    ],
    temperature=0.7,
    max_tokens=1000,
)

# Exibe a resposta
print(response.choices[0].message.content)

Claro! Vamos considerar um exemplo prático de uso de uma API em um contexto de um modelo de linguagem grande (LLM).

### Exemplo: Usando a API do OpenAI para gerar texto

Suponha que você deseja criar um aplicativo que gera resumos de textos longos usando um modelo de linguagem como o GPT da OpenAI. Aqui está como você poderia fazer isso usando a API do OpenAI:

1. **Obtenção da chave da API**: Primeiro, você precisaria se inscrever no serviço da OpenAI e obter uma chave da API, que permitirá que você faça requisições ao modelo.

2. **Configuração do ambiente**: Você precisaria de um ambiente Python com a biblioteca `requests` instalada. Você pode instalá-la usando o pip:

   ```bash
   pip install requests
   ```

3. **Fazendo uma requisição à API**: Aqui está um exemplo de código que realiza uma requisição para gerar um resumo:

   ```python
   import requests

   # Defina sua chave da API
   api_key = 'sua_chave_da_api_aqui'

   # O texto que você deseja resumir
   texto_para_resumi

**Principais parâmetros:**
- `model`: modelo a ser utilizado (ex: gpt-4o, gpt-4o-mini, gpt-3.5-turbo)
- `messages`: lista de mensagens com roles (system, user, assistant)
- `temperature`: criatividade da resposta (0 a 2), dependendo da aplicação
- `max_tokens`: limite de tokens na resposta
- `response_format`: força o modelo a retornar dados em formato estruturado específico

Além de OpenAI, podemos utilizar outras LLMs como Claude ou Gemini. Também é possível usar LiteLLM, que possibilita variedade de LLMs em um único ambiente.

In [ ]:
# Claude (Anthropic)
from anthropic import Anthropic

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

response = client.messages.create(
    model="claude-3-sonnet-20240229",
    messages=[{"role": "user", "content": "..."}],
    max_tokens=1024,
)

# Gemini (Google)
import google.generativeai as genai

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-pro")
response = model.generate_content("...")

In [27]:
import pandas as pd
from pydantic import BaseModel
from typing import Literal

# Exemplo de uso de response_format com Structured Outputs


# Definir o schema com Pydantic
class Tendencia(BaseModel):
    nome: str
    descricao: str
    impacto: Literal["alto", "medio", "baixo"]
    prazo: Literal["curto", "medio", "longo"]


class TendenciasResponse(BaseModel):
    tendencias: list[Tendencia]


# Requisição para pensarmos em tendências de um mercado
response_temperature1 = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an automotive market analyst. "
                "Return ONLY valid JSON with the schema: "
                '{"tendencias": [{"nome": str, "descricao": str, '
                '"impacto": "alto|medio|baixo", "prazo": "curto|medio|longo"}]} '
                "No extra text, markdown, or code fences."
            ),
        },
        {
            "role": "user",
            "content": (
                "List the 10 main automotive trends in Brazil following the required JSON schema."
            ),
        },
    ],
    temperature=0.2,
    max_tokens=800,
)

response_temperature2 = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": ("You are an automotive market analyst. "),
        },
        {
            "role": "user",
            "content": (
                "List the 10 main automotive trends in Brazil following the required JSON schema."
            ),
        },
    ],
    response_format=TendenciasResponse,
    temperature=1.5,
    max_tokens=800,
)

LengthFinishReasonError: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=800, prompt_tokens=179, total_tokens=979, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [25]:
import json

data_temperature1 = response_temperature1.choices[0].message.content
data_temperature2 = response_temperature2.choices[0].message.content

# Tentar carregar JSON com tratamento de erro
try:
    payload1 = json.loads(data_temperature1)
    df_temperature1 = pd.json_normalize(payload1["tendencias"])
except json.JSONDecodeError as e:
    print(f"Erro ao decodificar JSON de temperature1: {e}")
    df_temperature1 = pd.DataFrame()

try:
    payload2 = json.loads(data_temperature2)
    df_temperature2 = pd.json_normalize(payload2["tendencias"])
except json.JSONDecodeError as e:
    print(f"Erro ao decodificar JSON de temperature2: {e}")
    print("Resposta recebida (primeiros 500 caracteres):")
    print(data_temperature2[:500])
    df_temperature2 = pd.DataFrame()

Erro ao decodificar JSON de temperature2: Expecting ',' delimiter: line 12 column 259 (char 681)
Resposta recebida (primeiros 500 caracteres):
{
  "tendencias": [
    {
      "nome": "Veículos Elétricos",
      "descricao": "A expansão da linha de veículos elétricos comerciais e particulares em resposta às crescentes preocupações ambientais.",
      "impacto": "alto",
      "prazo": "medio"
    },
    {
      "nome": "Conectividade",
      "descricao": "Interação e integração de veículos com estratégia em casa, por meio de aplicativos e sistemas integrados.",
      "impacto": "altamente amplia automaticamente as futuras infraestruturas


In [5]:
df_temperature1

,nome,descricao,impacto,prazo
0,Veículos Elétricos,Aumento na adoção de veículos elétricos devido...,alto,medio
1,Carros Conectados,Integração de tecnologia de conectividade nos ...,alto,medio
2,Mobilidade Urbana,"Crescimento de soluções de mobilidade urbana, ...",medio,curto
3,Autonomia Veicular,Desenvolvimento de tecnologias para veículos a...,alto,longo
4,Sustentabilidade,Foco em práticas sustentáveis na produção e op...,medio,medio
5,Carros Híbridos,Aumento na venda de veículos híbridos como uma...,medio,curto
6,Tecnologia de Assistência ao Motorista,Implementação de sistemas avançados de assistê...,alto,medio
7,Mudanças Regulatórias,Novas regulamentações ambientais e de seguranç...,alto,curto
8,Digitalização da Experiência do Cliente,"Adoção de plataformas digitais para vendas, se...",medio,curto
9,Aumento da Concorrência,Entrada de novas montadoras e startups no merc...,medio,medio


In [ ]:
df_temperature2

,nome,descricao,impacto,prazo
0,Eletrificação dos Veículos,Crescimento no mercado de veículos elétricos e...,alto,medio
1,Digitalização e Conectividade,"Integração de tecnologias de conectividade, in...",alto,curto
2,Veículos Autônomos,Pesquisas para desenvolvimento de veículos aut...,medio,longo
3,Foco em Sustentabilidade,Marcas estão adotando práticas mais ecológicas...,alto,longo
4,Recuperação e Crescimento Pós-Pandemia,O mercado automotivo está mostrando sinais de ...,alto,curto
5,Popularização dos SUVs,A preferência por SUVs está crescendo entre os...,alto,medio
6,Uso de Tecnologia de Assistência ao Motorista,Motoristas mostram maior interesse por localiz...,medio,curto
7,Unificação das Plataformas CPFs,Concentração no compartilhamento de melhores p...,medio,medio
8,Aumento do E-commerce de Automóveis,"Veio devido à pandemia, um aumento no comercio...",medio,curto
9,Mobilidade Servida como Serviço (MaaS),Modelos de compras de inicia-se a implementar ...,baixo,longo


## 3) Trabalhando em escala

**Batches** permitem agrupar várias requisições em um único envio assíncrono. A API processa cada item separadamente, mas você ganha preço reduzido (tarifa de batch) e não precisa segurar a execução do seu código local enquanto a inferência acontece. O fluxo básico: gerar um arquivo NDJSON com todas as requisições, enviar esse arquivo para a OpenAI e disparar um job de batch. Depois, consultar o status até que os resultados estejam prontos e baixá-los.

**Requisições assíncronas** (_async_) são úteis quando você quer disparar várias chamadas rápidas e continuar trabalhando enquanto espera o retorno. Em Python, usamos `asyncio` e o cliente assíncrono da OpenAI para mandar várias requisições em paralelo sem bloquear a thread principal.

In [7]:
import json
import time
from pathlib import Path

BATCH_COMPLETION_WINDOW = "24h"  # janela mínima exigida pela API no momento

batch_requests = [
    {
        "custom_id": "trend-macro-001",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You are an automotive market analyst. Return ONLY valid JSON "
                        "with keys: tendencias -> [{nome, descricao, impacto, prazo}]"
                    ),
                },
                {
                    "role": "user",
                    "content": "Liste 5 tendências automotivas no Brasil seguindo o schema solicitado.",
                },
            ],
            "temperature": 0.0,
            "max_tokens": 400,
        },
    },
    {
        "custom_id": "trend-macro-002",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You are an automotive market analyst. Return ONLY valid JSON "
                        "with keys: tendencias -> [{nome, descricao, impacto, prazo}]"
                    ),
                },
                {
                    "role": "user",
                    "content": "Liste 5 tendências automotivas na Europa seguindo o schema solicitado.",
                },
            ],
            "temperature": 0.0,
            "max_tokens": 400,
        },
    },
]

batch_file = Path("batch_requests.ndjson")
batch_file.write_text(
    "\n".join(json.dumps(item) for item in batch_requests), encoding="utf-8"
)
print(f"Arquivo de batch gerado em: {batch_file.resolve()}")

uploaded = client.files.create(file=batch_file.open("rb"), purpose="batch")
batch_job = client.batches.create(
    input_file_id=uploaded.id,
    endpoint="/v1/chat/completions",
    completion_window=BATCH_COMPLETION_WINDOW,
)
print("Batch criado:", batch_job.id)

batch_job = client.batches.retrieve(batch_job.id)

Arquivo de batch gerado em: C:\Mirow\Treinamento Python\APIs\Treinamento\batch_requests.ndjson
Batch criado: batch_6903a98366ac81909ba4f920e0855841


In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Carrega as variáveis do arquivo .env
load_dotenv()

# Inicializa o cliente
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# batch_id = "batch_6903a98366ac81909ba4f920e0855841"
batch_id = "batch_6903a98366ac81909ba4f920e0855841"

batch_obj = client.batches.retrieve(batch_id)

print(f"Status do batch {batch_id}: {batch_obj.status}")

if batch_obj.status == "completed":
    print("\n✅ Batch completo! Baixando resultados...\n")
    result_file_id = batch_obj.output_file_id
    file_response = client.files.content(result_file_id)

    # Processar linhas JSONL
    results = []
    for line in file_response.text.strip().split("\n"):
        result = json.loads(line)
        results.append(result)

    print(f"📦 Total de resultados: {len(results)}\n")

    for idx, result in enumerate(results):
        custom_id = result.get("custom_id")
        response_body = result.get("response", {}).get("body", {})

        # Extrair conteúdo da resposta
        message_content = (
            response_body.get("choices", [{}])[0].get("message", {}).get("content", "")
        )

        print(f"✓ Resultado {idx} ({custom_id}):")
        print(f"  {message_content[:100]}...")

Status do batch batch_6903a98366ac81909ba4f920e0855841: completed

✅ Batch completo! Baixando resultados...

📦 Total de resultados: 2

✓ Resultado 0 (trend-macro-001):
  ```json
{
  "tendencias": [
    {
      "nome": "Veículos Elétricos",
      "descricao": "Aumento na...
✓ Resultado 1 (trend-macro-002):
  ```json
{
  "tendencias": [
    {
      "nome": "Eletrificação de Veículos",
      "descricao": "Aum...
📦 Total de resultados: 2

✓ Resultado 0 (trend-macro-001):
  ```json
{
  "tendencias": [
    {
      "nome": "Veículos Elétricos",
      "descricao": "Aumento na...
✓ Resultado 1 (trend-macro-002):
  ```json
{
  "tendencias": [
    {
      "nome": "Eletrificação de Veículos",
      "descricao": "Aum...


## 4) Use case: classificação de reclamações no Reclame Aqui

### Project Structure

```
02. Classificador RA/
├── main.py                    # Main orchestrator
├── src/
│   └── scraper.py                 # Phase 1: Data collection
│   └──theme_discovery.py         # Phase 2: Theme discovery
│   └──classifier.py              # Phase 4: Classification
│   └──usage_tracker.py           # OpenAI API usage tracking
│   └──view_usage.py              # View usage statistics
│   └──config.py                  # Configuration
├── requirements.txt           # Dependencies
├── .env                       # Environment variables (create from .env.example)
├── input/
│   └── complaints_raw.json    # Scraped complaints (Phase 1 output)
└── output/
    ├── proposed_taxonomy.json        # Proposed categories (Phase 2 output)
    ├── curated_taxonomy.json         # Final taxonomy (Phase 3 input)
    ├── classification_results.json   # Classifications (Phase 4 output)
    └── openai_usage.json             # API usage log (auto-generated)

### Execução direta da pipeline no notebook
As células abaixo reutilizam os componentes do projeto para demonstrar cada fase sem sair do Jupyter.

#### Configuração: exibir estatísticas de uso da API

In [2]:
from pathlib import Path
import sys
import json
import pandas as pd

PROJECT_ROOT = Path("02. Classificador RA").resolve()
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

import config

# Nomes de arquivos e diretórios
config.DATA_DIR = str(PROJECT_ROOT / "data")
config.OUTPUT_DIR = str(PROJECT_ROOT / "output")
config.API_USAGE_LOG_FILE = str(Path(config.OUTPUT_DIR) / "openai_usage.json")

config.SHOW_API_USAGE = True
config.SHOW_API_USAGE_DETAILS = False

config.COMPLAINTS_FILE = str(Path(config.DATA_DIR) / "complaints_raw.json")
config.PROPOSED_TAXONOMY_FILE = str(Path(config.OUTPUT_DIR) / "proposed_taxonomy.json")
config.CURATED_TAXONOMY_FILE = str(Path(config.OUTPUT_DIR) / "curated_taxonomy.json")
config.CLASSIFICATION_RESULTS_FILE = str(
    Path(config.OUTPUT_DIR) / "classification_results.json"
)

Path(config.DATA_DIR).mkdir(parents=True, exist_ok=True)
Path(config.OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

complaints_path = Path(config.COMPLAINTS_FILE)
fallback_path = PROJECT_ROOT / "input" / "complaints_raw.json"


print(
    "Uso de API habilitado. Estatísticas serão gravadas em:", config.API_USAGE_LOG_FILE
)

Uso de API habilitado. Estatísticas serão gravadas em: C:\Mirow\Treinamento Python\APIs\Treinamento\02. Classificador RA\output\openai_usage.json


#### 1. Scraping das reclamações (ou carregamento da base local)

In [3]:
from scraper import ReclameAquiAPIExtractor, SELENIUM_AVAILABLE

complaints_path = Path(config.COMPLAINTS_FILE)
fallback_path = PROJECT_ROOT / "input" / "complaints_raw.json"

# Extrair reclamações do ReclameAqui
if SELENIUM_AVAILABLE:
    extractor = ReclameAquiAPIExtractor(config.RECLAME_AQUI_URL, delay=1.5)
    complaints = extractor.scrape_all_complaints(max_pages=20)
else:
    if fallback_path.exists():
        print(
            "Selenium não disponível. Reutilizando base salva em input/complaints_raw.json."
        )
    else:
        raise RuntimeError(
            "Nem Selenium nem arquivo local disponível para carregar as reclamações."
        )

with open(complaints_path, "w", encoding="utf-8") as fp:
    json.dump(complaints, fp, ensure_ascii=False, indent=2)

print(f"Reclamações disponíveis: {len(complaints)}")
pd.DataFrame(complaints)[["complaint_id", "complaint_title", "status"]].head()

Starting scrape: up to 20 pages
Fetching page 1...
Fetching page 1...
✓ Extracted 10 complaints from page 1
✓ Extracted 10 complaints from page 1
Fetching page 2...
Fetching page 2...
✓ Extracted 10 complaints from page 2
✓ Extracted 10 complaints from page 2
Fetching page 3...
Fetching page 3...
✓ Extracted 10 complaints from page 3
✓ Extracted 10 complaints from page 3
Fetching page 4...
Fetching page 4...
✓ Extracted 10 complaints from page 4
✓ Extracted 10 complaints from page 4
Fetching page 5...
Fetching page 5...
✓ Extracted 10 complaints from page 5
✓ Extracted 10 complaints from page 5
Fetching page 6...
Fetching page 6...
✓ Extracted 10 complaints from page 6
✓ Extracted 10 complaints from page 6
Fetching page 7...
Fetching page 7...
✓ Extracted 10 complaints from page 7
✓ Extracted 10 complaints from page 7
Fetching page 8...
Fetching page 8...
✓ Extracted 10 complaints from page 8
✓ Extracted 10 complaints from page 8
Fetching page 9...
Fetching page 9...
✓ Extracted 10 com

,complaint_id,complaint_title,status
0,COMPLAINT_FNftCrSvmjTVSTIi,Negativa de garantia para peça substituída ant...,Aguardando resposta
1,COMPLAINT_hMBfJpzQmQ46G4_k,Caminhão com [NOME] de Óleo e [NOME] em Garantia,Aguardando resposta
2,COMPLAINT_LeSwbKMr-EZHSWpw,[NOME] 2024/2025 com problemas na marcha e dir...,Aguardando resposta
3,COMPLAINT_A5atRAmWVxU9E32k,Absurdo no preço do banco e durabilidade infer...,Aguardando resposta
4,COMPLAINT_E5euaQanLCg2t2et,Solicitação de [NOME] e Revisão da Análise de ...,Aguardando resposta


### Configuração de Agentes com YAML

Para facilitar a manutenção e permitir ajustes rápidos sem alterar código, externalizamos as configurações dos agentes LLM em arquivos YAML.

#### Estrutura de Diretórios
```
02. Classificador RA/
├── agents/
│   ├── theme_discovery.yaml        # Agente de descoberta de temas
│   └── complaint_classifier.yaml   # Agente de classificação
└── src/
    ├── agent_loader.py             # Carrega configs YAML
    ├── theme_discovery.py          # Usa theme_discovery.yaml
    └── classifier.py               # Usa complaint_classifier.yaml
```

#### Exemplo de Arquivo YAML: `agents/theme_discovery.yaml`

```model: "gpt-4-turbo"
temperature:  0.3
max_tokens: 2000

system_prompt: |
  You are an expert in analyzing customer complaints and identifying patterns.
  Your task is to propose a taxonomy of categories that best represents the themes
  present in a set of complaints.

user_prompt_template: |
  Based on the following complaints, propose between {min_categories} and {max_categories}
  thematic categories that would best organize them.
  
  Complaints:
  {complaints_sample}
  
  Return ONLY valid JSON with this exact structure:
  {{
    "proposed_categories": [
      {{
        "category_name": "Category Name",
        "category_description": "Detailed description",
        "keywords": ["keyword1", "keyword2"]
      }}
    ]
  }}
```

#### Como Usar no Código

```python
# src/agent_loader.py
import yaml
from pathlib import Path

def load_agent_config(agent_name: str) -> dict:
    config_path = Path(__file__).parent.parent / "agents" / f"{agent_name}.yaml"
    with open(config_path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

# src/theme_discovery.py
from agent_loader import load_agent_config

class ThemeDiscovery:
    def __init__(self, api_key: str):
        # Carrega configuração do YAML
        self.config = load_agent_config('theme_discovery')
        self.model = self.config['model']
        self.temperature = self.config['temperature']
        self.system_prompt = self.config['system_prompt']
        
    def generate_taxonomy(self, complaints):
        response = client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.config['user_prompt_template'].format(
                    min_categories=5,
                    max_categories=15,
                    complaints_sample=complaints
                )}
            ]
        )
        return response
```

#### Vantagens desta Abordagem

1. **Separação de Responsabilidades**: Lógica de código separada de configuração
2. **Iteração Rápida**: Ajuste prompts sem tocar no código Python
3. **Controle de Versão**: Fácil comparar mudanças nos prompts via Git
4. **Experimentação**: Teste múltiplas versões de prompts facilmente
5. **Manutenibilidade**: Prompts longos ficam mais legíveis em YAML

#### Reload de Configs no Notebook

Se você alterar um arquivo YAML durante a execução do notebook, force o reload:

```python
import importlib
import sys

if 'agent_loader' in sys.modules:
    importlib.reload(sys.modules['agent_loader'])
if 'theme_discovery' in sys.modules:
    importlib.reload(sys.modules['theme_discovery'])
```

In [5]:
# Forçar reload dos módulos para pegar alterações no YAML
import importlib
import sys

# Recarregar módulos se já estiverem em cache
if "agent_loader" in sys.modules:
    importlib.reload(sys.modules["agent_loader"])
    print("✓ agent_loader recarregado")

if "theme_discovery" in sys.modules:
    importlib.reload(sys.modules["theme_discovery"])
    print("✓ theme_discovery recarregado")

if "classifier" in sys.modules:
    importlib.reload(sys.modules["classifier"])
    print("✓ classifier recarregado")

print("\nVerificando YAML atual:")
from agent_loader import load_agent_config

print(f"theme_discovery model: {load_agent_config('theme_discovery').get('model')}")

✓ agent_loader recarregado

Verificando YAML atual:
theme_discovery model: gpt-4-turbo


#### 2. Geração automática de categorias com OpenAI

In [6]:
from theme_discovery import ThemeDiscovery

if not config.OPENAI_API_KEY:
    raise RuntimeError(
        "Defina OPENAI_API_KEY no arquivo .env antes de gerar a taxonomia."
    )

# Classe do Theme Discovery
discovery = ThemeDiscovery(config.OPENAI_API_KEY, track_usage=True)

# Verificando se iremos trackear utilização da API
if discovery.tracker:
    discovery.tracker.start_session(
        "Notebook - Fase 2 (Theme Discovery)", discovery.model
    )

complaints = discovery.load_complaints(config.COMPLAINTS_FILE)
sample = discovery.sample_complaints(complaints, config.SAMPLE_SIZE_FOR_DISCOVERY)
taxonomy_payload = discovery.generate_taxonomy(sample)
discovery.save_taxonomy(
    taxonomy_payload, config.PROPOSED_TAXONOMY_FILE
)  # Salvar taxonomia proposta

print(f"Categorias propostas: {len(taxonomy_payload['proposed_categories'])}")
taxonomy_df = pd.DataFrame(taxonomy_payload["proposed_categories"])
display(taxonomy_df[["category_name", "category_description"]])

if discovery.tracker:
    session_data = discovery.tracker.end_session()
    if session_data:
        print("\nUso da API - Fase 2 (Theme Discovery):")
        print(
            json.dumps(
                {
                    "total_tokens": session_data["total_tokens"],
                    "input_tokens": session_data["total_input_tokens"],
                    "output_tokens": session_data["total_output_tokens"],
                    "estimated_cost_usd": session_data["estimated_cost_usd"],
                    "api_calls": len(session_data["calls"]),
                },
                indent=2,
                ensure_ascii=False,
            )
        )

Categorias propostas: 6


,category_name,category_description
0,Warranty and Part Replacement Issues,Customers express dissatisfaction due to denie...
1,Quality and Durability Concerns,Customers report problems with the quality and...
2,Service and Support Challenges,Customers encounter poor service experiences i...
3,Technology and Connectivity Issues,Issues related to the functionality and user a...
4,Cost and Pricing Disputes,"Customers feel certain costs are unjustified, ..."
5,Recall Management Issues,Customers are dissatisfied with how recalls ar...



Uso da API - Fase 2 (Theme Discovery):
{
  "total_tokens": 18308,
  "input_tokens": 17731,
  "output_tokens": 577,
  "estimated_cost_usd": 0.1946,
  "api_calls": 1
}


#### 3. Classificação das reclamações com a taxonomia definida

In [10]:
from classifier import ComplaintClassifier

if not config.OPENAI_API_KEY:
    raise RuntimeError(
        "Defina OPENAI_API_KEY no arquivo .env antes de classificar as reclamações."
    )

taxonomy_path = Path(config.CURATED_TAXONOMY_FILE)
if not taxonomy_path.exists():
    print("Taxonomia curada não encontrada. Usando proposta recém-gerada.")
    taxonomy_path = Path(config.PROPOSED_TAXONOMY_FILE)

classifier = ComplaintClassifier(config.OPENAI_API_KEY, track_usage=True)
if classifier.tracker:
    classifier.tracker.start_session(
        "Notebook - Fase 3 (Classificação)", classifier.model
    )

taxonomy = classifier.load_taxonomy(str(taxonomy_path))
complaints = classifier.load_complaints(config.COMPLAINTS_FILE)
results = classifier.classify_all(
    complaints=complaints, taxonomy=taxonomy, use_batch=True
)
summary = classifier.generate_summary(results)

output_payload = {
    "taxonomy_used": taxonomy,
    "classification_results": results,
    "summary": summary,
}
with open(config.CLASSIFICATION_RESULTS_FILE, "w", encoding="utf-8") as fp:
    json.dump(output_payload, fp, ensure_ascii=False, indent=2)

print(f"Reclamações classificadas nesta célula: {len(results)}")
display(pd.DataFrame(summary["category_distribution"]))

if classifier.tracker:
    session_data = classifier.tracker.end_session()
    if session_data:
        print("\nUso da API - Fase 3 (Classificação):")
        print(
            json.dumps(
                {
                    "total_tokens": session_data["total_tokens"],
                    "input_tokens": session_data["total_input_tokens"],
                    "output_tokens": session_data["total_output_tokens"],
                    "estimated_cost_usd": session_data["estimated_cost_usd"],
                    "api_calls": len(session_data["calls"]),
                },
                indent=2,
                ensure_ascii=False,
            )
        )

Classifying complaints: 100%|██████████| 20/20 [03:11<00:00,  9.57s/it]

Reclamações classificadas nesta célula: 200


,category,count,percentage
0,Quality and Durability Concerns,78,39.0
1,Warranty and Part Replacement Issues,54,27.0
2,Service and Support Challenges,23,11.5
3,Technology and Connectivity Issues,20,10.0
4,OTHER,12,6.0
5,Recall Management Issues,10,5.0
6,Cost and Pricing Disputes,3,1.5



Uso da API - Fase 3 (Classificação):
{
  "total_tokens": 30160,
  "input_tokens": 22507,
  "output_tokens": 7653,
  "estimated_cost_usd": 0.008,
  "api_calls": 20
}


### Insights Adicionais e Análises Possíveis

#### Análises Quantitativas
- **Evolução temporal**: Identificar tendências de reclamações ao longo do tempo (sazonalidade, picos)
- **Análise de sentimento**: Classificar reclamações por intensidade emocional (urgente, frustrado, neutro)
- **Taxa de resolução**: Correlacionar categorias com status de resolução para identificar gargalos

#### Análises Qualitativas
- **Extração de palavras-chave**: Identificar termos mais frequentes por categoria usando TF-IDF ou embeddings
- **Clustering automático**: Agrupar reclamações similares para descobrir subcategorias não mapeadas
- **Análise de causa raiz**: Usar LLMs para identificar causas subjacentes comuns em cada categoria

#### Análises Comparativas
- **Benchmarking**: Comparar padrões de reclamações entre diferentes modelos de veículos ou concessionárias
- **Análise competitiva**: Contrastar taxonomia e volumes com outras montadoras premium

#### Automatização e Monitoramento
- **Alertas inteligentes**: Detectar anomalias (ex: aumento súbito de reclamações em categoria específica)
- **Dashboard em tempo real**: Visualizar distribuição de categorias e métricas de classificação
- **Sistema de priorização**: Ranquear reclamações por urgência e impacto potencial usando scoring multi-critério